In [100]:
import os
import random
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
import math
random.seed(datetime.now())

In [101]:
database='./data_new'
dirs=['negative','positive','polluted']
samples={}

In [102]:
def getGroup(str):
    return str[:str.find("_")]

In [103]:
def getDateStr(str):
    if '/' in str:
        str=str[str.rfind('/'):]
    return str[str.find("_")+1:str.find(".")]

In [104]:
def getDate(str):
    return datetime.strptime(getDateStr(str), "%Y%m%d").date()

In [105]:
samples={}
print("+++ Start building info +++")
for r,ds,fs in os.walk(database):
    for f in fs:
        path=os.path.join(r,f)
        group=getGroup(f)
        if 'txt' not in f and 'jpg' in f:
            if len(group)!=1 and group!='discrete':
                if group not in samples:
                    samples[group]={'files':[],'result':0,'duration':0,'length':0,'type':'sample','firstdate':None}
                samples[group]['files'].append(path)
            else:
                samples[f]={'files':[path],'result':0,'duration':1,'length':1,'type':'discrete','firstdate':None}
                if 'positive' in path:
                    samples[f]['result']=1
                if 'polluted' in path:
                    samples[f]['result']=2
print("+++ Building info done +++")

+++ Start building info +++
+++ Building info done +++


In [107]:

print("+++ Start sorting +++")
for k,v in samples.items():
    samples[k]['files']=sorted(samples[k]['files'],key=lambda x:getDateStr(x))
print("+++ Sorting done +++")



+++ Start sorting +++
+++ Sorting done +++


In [108]:
print("+++ Start computing duration and length +++")
for k,v in (samples.items()):
    if samples[k]['type']=="sample":
        samples[k]['duration']=(getDate(samples[k]['files'][-1])-getDate(samples[k]['files'][0])).days+1
        samples[k]['length']=len(samples[k]['files'])
        samples[k]['firstdate']=getDate(samples[k]['files'][0])
        if 'positive' in samples[k]['files'][-1]:
            samples[k]['result']=1
        if 'polluted' in samples[k]['files'][-1]:
            samples[k]['result']=2
print("+++ Computing duration and length done +++")

+++ Start computing duration and length +++
+++ Computing duration and length done +++


In [109]:
negative={}
positive={}
polluted={}
for k,v in (samples.items()):
    if samples[k]['result']==0:
        negative[k]=samples[k]
    elif samples[k]['result']==1:
        positive[k]=samples[k]
    elif samples[k]['result']==2:
        polluted[k]=samples[k]

In [110]:
print(len(negative))
print(len(positive))
print(len(polluted))

1430
182
124


In [111]:
vali_negative_groups=random.sample(set(negative),100)
vali_positive_groups=random.sample(set(positive),100)
vali_polluted_groups=random.sample(set(polluted),100)
vali_groups=vali_negative_groups+vali_positive_groups+vali_polluted_groups

In [112]:
train_negative=[]
train_positive=[]
train_polluted=[]
jail_negative=[]
jail_positive=[]
jail_polluted=[]
vali_negative=[]
vali_positive=[]
vali_polluted=[]

In [113]:
for k,v in samples.items():
    if k not in vali_groups:
        for f in v['files']:
            if 'negative' in f:
                train_negative.append(f)
            elif 'positive' in f:
                train_positive.append(f)
            elif 'polluted' in f:
                train_polluted.append(f)
            else:
                print("Type error:",f)
    else:
        vali_file=v['files'][-1]
        if 'negative' in vali_file:
            vali_negative.append(vali_file)
        elif 'positive' in vali_file:
            vali_positive.append(vali_file)
        elif 'polluted' in vali_file:
            vali_polluted.append(vali_file)
        files=v['files'][:-1]
        remainder_len=len(files)
        train_len=math.ceil(remainder_len*0.8)
        #print("Class",v['result'],"Remain:",remainder_len,"Train:",train_len,"Jail:",remainder_len-train_len,k)
        for i,f in enumerate(files):
            if i<=train_len:
                if 'negative' in f:
                    train_negative.append(f)
                elif 'positive' in f:
                    train_positive.append(f)
                elif 'polluted' in f:
                    train_polluted.append(f)
            else:
                if 'negative' in f:
                    jail_negative.append(f)
                elif 'positive' in f:
                    jail_positive.append(f)
                elif 'polluted' in f:
                    jail_polluted.append(f)

In [114]:
print(len(train_negative),len(jail_negative),len(vali_negative))
print(len(train_positive),len(jail_positive),len(vali_positive))
print(len(train_polluted),len(jail_polluted),len(vali_polluted))

14705 41 100
541 19 100
868 38 100


In [115]:
with open('./mapping/190408_newdata_train_cnn_mapping.csv','w') as file:
    file.write("file_path,label\n")
    for f in train_negative:
        file.write(f+",0\n")
    for f in train_positive:
        file.write(f+",1\n")
    for f in train_polluted:
        file.write(f+",2\n")
with open('./mapping/190408_newdata_train_yolo9000_mapping.csv','w') as file:
    file.write("file_path,label\n")
    for f in train_negative:
        file.write(f+".txt,0\n")
    for f in train_positive:
        file.write(f+".txt,1\n")
    for f in train_polluted:
        file.write(f+".txt,2\n")
with open('./mapping/190408_newdata_vali_cnn_mapping.csv','w') as file:
    file.write("file_path,label\n")
    for f in vali_negative:
        file.write(f+",0\n")
    for f in vali_positive:
        file.write(f+",1\n")
    for f in vali_polluted:
        file.write(f+",2\n")
with open('./mapping/190408_newdata_vali_yolo9000_mapping.csv','w') as file:
    file.write("file_path,label\n")
    for f in vali_negative:
        file.write(f+".txt,0\n")
    for f in vali_positive:
        file.write(f+".txt,1\n")
    for f in vali_polluted:
        file.write(f+".txt,2\n")